In [1]:
import numpy as np 
import pandas as pd
#Random Forest Classification
# text preprocessing


from sklearn.feature_extraction.text import TfidfVectorizer
import nltk,re 
from nltk.stem import WordNetLemmatizer
# model building
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
#RNN MODEL
# text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

# model building
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN,Flatten, Dropout,Embedding,Bidirectional,LSTM,Flatten,MaxPooling1D
from keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report ,accuracy_score

# split data into train and test set
from sklearn.model_selection import train_test_split

#transfer learning RNN
from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.utils import np_utils


from gensim import models
import collections

In [2]:
import tensorflow as tf

# Load and explore data

In [2]:
!gdown --id "1al0kSfJZbN1ham-IgOlClaJLXNy74val"

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1al0kSfJZbN1ham-IgOlClaJLXNy74val
To: /content/author_data.csv
100% 3.30M/3.30M [00:00<00:00, 104MB/s]


In [3]:
data_path = "/content/author_data.csv"
data_frame = pd.read_csv(data_path)
data_frame 

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [4]:
df = pd.DataFrame.copy(data_frame)
df.drop(columns=['id'],inplace=True)
df = df.drop_duplicates()

authors=list(df['author'].unique())
authors

['EAP', 'HPL', 'MWS']

In [5]:
df['author'].value_counts()

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

# Data Preprocessing

In [6]:
# 'EAP'-> 0  'HPL'-> 1 'MWS'-> 2 
df['author'].replace(authors,[0,1,2], inplace=True)

In [7]:
df = df.sample(frac = 1)

In [8]:
X = df['text']
Y = df['author']
print(X.shape)
print(Y.shape)

(19579,)
(19579,)


In [9]:
X.values

array(['They found Gilman on the floor of his queerly angled old garret room long before dawn, for the terrible cry had brought Desrochers and Choynski and Dombrowski and Mazurewicz at once, and had even wakened the soundly sleeping Elwood in his chair.',
       'I found that the wind was northeast and must have driven me far from the coast from which I had embarked.',
       'But he requested my attention particularly, and with an air of mysterious sagacity, to a thick octavo, written in barbarous Latin by one Hedelin, a Frenchman, and having the quaint title, "Duelli Lex Scripta, et non; aliterque."',
       ...,
       'To describe their exact nature is impossible for one unversed in music.',
       'Scarcely less savage was the "Lollipop," which thus discoursed: "Some individual, who rejoices in the appellation \'Oppodeldoc,\' to what low uses are the names of the illustrious dead too often applied has enclosed us some fifty or sixty verses commencing after this fashion: Achilles\'

In [10]:
vocab = set([ word for sentence in X.values for word in sentence.split()])
vocab_length = len(vocab)

train_x_non, train_y = X[:int(.75*len(X))].values,Y[:int(.75*len(Y))].values
test_x_non , test_y  = X[int(.75*len(X)):].values,Y[int(.75*len(Y)):].values
print(train_x_non.shape, train_y.shape,'\n',test_x_non .shape, test_y.shape)

(14684,) (14684,) 
 (4895,) (4895,)


### text processing

In [11]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#stopword removal and lemmatization
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [13]:
#text pre processing on training data
train_X = []
test_X =  []
train_x_non = list(train_x_non)
for i in range(0, len(train_x_non)):
  review = re.sub('[^a-zA-Z]', ' ', train_x_non[i])
  review = review.lower()
  review = review.split()
  review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
  review = ' '.join(review)
  train_X.append(review)

#text pre processing on test data
for i in range(0, len((test_x_non))):
  review = re.sub('[^a-zA-Z]', ' ', test_x_non[i])
  review = review.lower()
  review = review.split()
  review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
  review = ' '.join(review)
  test_X.append(review)

# Models

## Random Forest Classification 

### TF-IDF Vectorizer

In [25]:
#tf idf
tf_idf = TfidfVectorizer()
X_train_tf = tf_idf.fit_transform(train_X)
X_train_tf = tf_idf.transform(train_X)
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 14684, n_features: 19688


In [26]:
 X_test_tf = tf_idf.transform(test_X)
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 4895, n_features: 19688


### Model building

In [68]:
# RandomForestClassifier
random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(X_train_tf, train_y)

RandomForestClassifier()

In [69]:
#predicted y
y_pred = random_forest_classifier.predict(X_test_tf)
print(metrics.classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.83      0.74      1980
           1       0.75      0.61      0.68      1401
           2       0.77      0.66      0.71      1514

    accuracy                           0.71      4895
   macro avg       0.73      0.70      0.71      4895
weighted avg       0.72      0.71      0.71      4895



## RNN model 


###Text Pre-Processing

* Tokenization
* Text Encoding
* Padding
* one hot encoding


In [32]:
# Tokenization
max_length = max([len(i.split(" ")) for i in X]) #maximum number of words in a single sentence

text_tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True, split=' ', num_words=max_length, oov_token="<pad>", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
text_tokenizer.fit_on_texts(train_X)

In [33]:
#  Text Encoding
encoded_train = text_tokenizer.texts_to_sequences(train_X)
encoded_test = text_tokenizer.texts_to_sequences(test_X)
print(encoded_train[0:2])

[[27, 648, 240, 1, 1, 14, 1, 64, 34, 1, 254, 1, 198, 1, 1, 1, 1, 11, 1, 1, 1, 1, 745], [27, 215, 1, 19, 1, 54, 1, 1]]


In [34]:
# Padding

padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [35]:
def One_hot_encoding(index : int):
  arr=np.zeros(3)
  arr[index] = 1
  return arr

In [36]:
train_encoded_y = np.array([One_hot_encoding(i) for i in train_y ] )
test_encoded_y  = np.array([One_hot_encoding(i) for i in test_y ] )
test_encoded_y.shape

(4895, 3)

### Simple RNN model  building

In [148]:
vocab_size = vocab_length

# define the model
model = Sequential()
model.add(Embedding(max_length+1, 40))

model.add(Bidirectional(LSTM(128)))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(64,activation='relu'))
model.add(Dense(3, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, None, 40)          34480     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              53760     
 l)                                                              
                                                                 
 dense_24 (Dense)            (None, 64)                8256      
                                                                 
 dense_25 (Dense)            (None, 3)                 195       
                                                                 
Total params: 96,691
Trainable params: 96,691
Non-trainable params: 0
_________________________________________________________________
None


In [149]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# fit the model
model.fit(x=padded_train,y=train_encoded_y,epochs=100,validation_data=(padded_test,test_encoded_y), verbose=1,batch_size=48)

Epoch 1/100
306/306 [==============================] - 23s 62ms/step - loss: 0.8795 - accuracy: 0.5769 - val_loss: 0.6888 - val_accuracy: 0.6995
Epoch 2/100
306/306 [==============================] - 18s 60ms/step - loss: 0.6547 - accuracy: 0.7252 - val_loss: 0.6736 - val_accuracy: 0.7068
Epoch 3/100
306/306 [==============================] - 18s 60ms/step - loss: 0.6241 - accuracy: 0.7377 - val_loss: 0.6732 - val_accuracy: 0.7066
Epoch 4/100
306/306 [==============================] - 18s 59ms/step - loss: 0.6059 - accuracy: 0.7458 - val_loss: 0.6760 - val_accuracy: 0.7109
Epoch 5/100
306/306 [==============================] - 19s 61ms/step - loss: 0.5880 - accuracy: 0.7518 - val_loss: 0.6875 - val_accuracy: 0.7073
Epoch 6/100
306/306 [==============================] - 19s 62ms/step - loss: 0.5739 - accuracy: 0.7575 - val_loss: 0.6983 - val_accuracy: 0.7017
Epoch 7/100
306/306 [==============================] - 18s 60ms/step - loss: 0.5619 - accuracy: 0.7615 - val_loss: 0.7255 - val_ac

KeyboardInterrupt: ignored

In [150]:
preds = (model.predict(padded_test) > 0.5).astype("int32")

153/153 [==============================] - 4s 24ms/step


In [151]:
def c_report(y_true, y_pred):
   print("Classification Report")
   print(classification_report(y_true, y_pred))
   acc_sc = accuracy_score(y_true, y_pred)
   print("Accuracy : "+ str(acc_sc))
   return acc_sc


c_report(test_encoded_y, preds)


Classification Report
              precision    recall  f1-score   support

           0       0.73      0.68      0.70      1980
           1       0.76      0.59      0.66      1401
           2       0.70      0.69      0.69      1514

   micro avg       0.72      0.66      0.69      4895
   macro avg       0.73      0.65      0.69      4895
weighted avg       0.73      0.66      0.69      4895
 samples avg       0.66      0.66      0.66      4895

Accuracy : 0.6576098059244126


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6576098059244126

### transfer learning RNN 

In [ ]:
!pip install gdown

# to upgrade
!pip install --upgrade gdown

In [155]:
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:13<00:00, 120MB/s] 


In [156]:
!gzip -d '/content/GoogleNews-vectors-negative300.bin.gz'

In [15]:
INPUT_FILE  = "/content/GoogleNews-vectors-negative300.bin"
NUM_WORDS   = max_length
EMBED_SIZE  = 300

In [16]:
weights = models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True)

In [37]:
word_to_number = {}
number_to_word = {}

for i,j in enumerate(vocab):
  word_to_number[i] = j
  number_to_word[j] = i

In [38]:
# Load weights for the embedding layer 
word2vec_embedding_weights = np.zeros((vocab_length+1, 300))

for i in range(max_length):
    word = word_to_number[i+1]
    if word in weights:
        word2vec_embedding_weights[i+1,:] = weights[word]

In [64]:
def create_model(initial_weights=None, trainable=True):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_length+1, 
                    output_dim=initial_weights.shape[1], 
                    weights=[initial_weights],
                    trainable=trainable))

    model.add(Bidirectional(LSTM(128,return_sequences=True)))
    model.add(Conv1D(64,8))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    return model

In [65]:
transfer_RNN_model = create_model(initial_weights=word2vec_embedding_weights, trainable=True)
transfer_RNN_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [66]:
transfer_RNN_model_history = transfer_RNN_model.fit(x=padded_train,y=train_encoded_y,epochs=100,validation_data=(padded_test,
test_encoded_y),batch_size=16, verbose=1)

Epoch 1/100
918/918 [==============================] - 81s 85ms/step - loss: 0.8059 - accuracy: 0.6367 - val_loss: 0.6962 - val_accuracy: 0.7001
Epoch 2/100
918/918 [==============================] - 76s 83ms/step - loss: 0.6681 - accuracy: 0.7153 - val_loss: 0.6897 - val_accuracy: 0.7005
Epoch 3/100
918/918 [==============================] - 76s 83ms/step - loss: 0.6153 - accuracy: 0.7412 - val_loss: 0.6990 - val_accuracy: 0.6962
Epoch 4/100
918/918 [==============================] - 76s 83ms/step - loss: 0.5582 - accuracy: 0.7662 - val_loss: 0.7363 - val_accuracy: 0.7019
Epoch 5/100
918/918 [==============================] - 75s 82ms/step - loss: 0.4857 - accuracy: 0.7975 - val_loss: 0.8452 - val_accuracy: 0.6919
Epoch 6/100
918/918 [==============================] - 76s 83ms/step - loss: 0.4066 - accuracy: 0.8330 - val_loss: 0.9505 - val_accuracy: 0.6721
Epoch 7/100
149/918 [===>..........................] - ETA: 56s - loss: 0.2882 - accuracy: 0.8838

KeyboardInterrupt: ignored

In [68]:
# evaluate model
score = transfer_RNN_model.evaluate(padded_test, test_encoded_y, verbose=0)
print("Validation score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

Validation score: 1.057, accuracy: 0.678


### hybrid approach 



> pre-trained embeddings with embeddings learned on the fly



In [76]:
import tensorflow.keras.backend as K
class Attention(tf.keras.layers.Layer):
  def __init__(self,trainable,**kwargs):
    super(Attention,self).__init__(**kwargs)
    self.trainable = trainable

  def build(self,input_shape):
    self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                            initializer='random_normal', trainable=self.trainable)
    self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                            initializer='zeros', trainable=self.trainable)        
    super(Attention, self).build(input_shape)

  def call(self,x):
    # Alignment scores. Pass them through tanh function
    e = K.tanh(K.dot(x,self.W)+self.b) # h > x
    # Remove dimension of size 1
    e = K.squeeze(e, axis=-1)   
    # Compute the weights
    alpha = K.softmax(e)
    # Reshape to tensorFlow format
    alpha = K.expand_dims(alpha, axis=-1)
    # Compute the context vector
    context_r = x * alpha
    context = K.sum(context_r, axis=1)
    return context,context_r

In [95]:
input = tf.keras.layers.Input(shape = max_length,name='input')


x = Embedding(input_dim=vocab_length+1, 
                    output_dim=word2vec_embedding_weights.shape[1], 
                    weights=[word2vec_embedding_weights],
                    trainable=False)(input)

_,x = Attention(trainable=True)(x)
x = Flatten()(x)
x = Dense(64)(x)

y = Embedding(input_dim=vocab_length+1, 
                    output_dim=40, 
                    trainable=True)(input)

_,y = Attention(trainable=True)(y)
y = Flatten()(y)
y = Dense(64)(y)
concatenated = tf.keras.layers.concatenate(inputs=[x,y])

z = tf.keras.layers.Dense(128, activation='relu')(concatenated)
out = tf.keras.layers.Dense(3,activation='softmax')(z)

newmodel = tf.keras.models.Model(inputs=input, outputs=out)

newmodel.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [96]:
transfer_RNN_model_history = newmodel.fit(x=padded_train,y=train_encoded_y,epochs=100,validation_data=(padded_test,
test_encoded_y),batch_size=32, verbose=1)

Epoch 1/100
459/459 [==============================] - 6s 12ms/step - loss: 1.0596 - accuracy: 0.4353 - val_loss: 0.9746 - val_accuracy: 0.5073
Epoch 2/100
459/459 [==============================] - 5s 11ms/step - loss: 0.9013 - accuracy: 0.5746 - val_loss: 0.8708 - val_accuracy: 0.5877
Epoch 3/100
459/459 [==============================] - 5s 11ms/step - loss: 0.7648 - accuracy: 0.6613 - val_loss: 0.8541 - val_accuracy: 0.5980
Epoch 4/100
459/459 [==============================] - 5s 11ms/step - loss: 0.6800 - accuracy: 0.7016 - val_loss: 0.8560 - val_accuracy: 0.6084
Epoch 5/100
459/459 [==============================] - 5s 11ms/step - loss: 0.5973 - accuracy: 0.7480 - val_loss: 0.8218 - val_accuracy: 0.6474
Epoch 6/100
459/459 [==============================] - 5s 11ms/step - loss: 0.5228 - accuracy: 0.7819 - val_loss: 0.8300 - val_accuracy: 0.6605
Epoch 7/100
459/459 [==============================] - 5s 11ms/step - loss: 0.4752 - accuracy: 0.8064 - val_loss: 0.8518 - val_accuracy:

KeyboardInterrupt: ignored

In [99]:
# evaluate model
score = newmodel.evaluate(padded_test, test_encoded_y, verbose=0)
print("Validation score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

Validation score: 2.669, accuracy: 0.626
